# M1 MEG - UE 5 A2B - TP5
## Pipeline d'analyse RNA-seq - Partie 1

L'objectif de ce TP est de faire tourner les premières étapes du pipeline d'analyse RNA-seq : 
- Comprendre les différentes étapes
- Comprendre le lien entre les étapes
- Analyser les résultats

**Quelques rappels** : 
- Un ordinateur / serveur de calcul est complètement stupide : il fera *exactement* ce que vous avez demandé de faire (ni plus ni moins).
- 99% des problèmes en informatique sont situés entre l'écran et la chaise.

**Mode de fonctionnement:**

- Les commandes que vous aurez à executer vous sont données dans le notebook. 
- À vous de les exécuter dans l'ordre
- Inutile de tout faire tourner d'un seul coup, un objectif essentiel est de *comprendre* ce que font les programmes, ce qu'ils retournent et *analyser* les résultats.

**Emplacements des Données :**
- Données Brutes : `/srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/`
- Annotation du Génome : `/srv/data/Genomes/Mmu/GRCm39/extracted/genome_annotation-M37.gtf`
- Index du Génome : `/srv/data/Genomes/Mmu/GRCm39/indexes_upto49bases/`

**Configuration de l'environnement**
Les outils requis sont pré-installés dans l'environnement meg-m1-ue5-unix2 dans lequel vous êtes :
- FastQC (v0.12.1) - Contrôle qualité
- MultiQC (v1.13) - Rapports agrégés
- fastp (v0.23.1) - Prétraitement des reads
- STAR (v2.7.11a) - Alignement des reads
- samtools (v1.18) - Manipulation des fichiers BAM


# 1. Prépration

## Création des répertoires de sortie

Comme indiqué ci-dessus, un ordinateur est complètement stupide et ne fera que ce que vous lui demandez de faire. 
Par exemple, si vous demandez à un programme d'enregistrer des fichiers de sortie dans un dossier *qui n'existe pas*, il va y avoir un soucis.

Il est donc important de *préparer* les dossiers de sortie et de les créer.

Nous allons créer un dossier principal où tous les résultats seront enregistrés, puis des sous-dossiers pour chaque étape / outil.

**Conseil** : les meilleurs amis d'un.e bioinformaticien.ne reste le papier et le stylo : ayez toujours une feuille / un cahier à côté de vous où vous allez *plannifier* vos analyses : quelles étapes, quels outils, etc...

In [1]:
mkdir -p Analysis_chr7/1-beforeTrimming/fastqc
mkdir -p Analysis_chr7/1-beforeTrimming/multiqc

mkdir -p Analysis_chr7/2-afterTrimming/fastp
mkdir -p Analysis_chr7/2-afterTrimming/fastqc
mkdir -p Analysis_chr7/2-afterTrimming/multiqc

mkdir -p Analysis_chr7/3-mapping/bam
mkdir -p Analysis_chr7/3-mapping/multiqc

On a créé un dossier principal (`Analysis_chr7`), trois sous-dossiers (`1-beforeTrimming`, `2-afterTrimming`et `3-mapping`), ainsi que des dossier pour chaque outil.

**Questions** : 
- À quoi sert l'option `-p` du programme `mkdir` ?
- Quel peut être l'intérêt de numéroter les dossiers ?

## Vérification des fichiers de données initiaux

Grâce aux sommes de contrôle, on peut vérifier l'intégrité de nos données de base.

In [ ]:
ls -lh /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/

In [ ]:
md5sum /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/SRX1589831_chr7_R1.fastq.gz

In [ ]:
head /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/md5sum.txt

In [ ]:
md5sum /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/*.fastq.gz

# 2. Contrôle qualité

## FastQC

La première étape d'un pipeline (RNA-seq ou autre) est de vérifier la qualité des données. L'outil `fastqc` permet de calculer un certain nombre de statistiques sur des fichiers fastq.

Commencez par jeter un œil aux différentes options du programme (en général, taper le nom du programme avec l'option `-h` ou `--help` permet de voir la liste des options)

In [ ]:
fastqc -h

On décide de faire tourner l'outils sur *tous* les fichiers fastq contenus dans le dossier contenant les données.

In [ ]:
fastqc -t 5 -o ./Analysis_chr7/1-beforeTrimming/fastqc/ \
    /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/*.fastq.gz

**Question** :
- Combien de fichiers de sortie *par fichier fastq* obtenez-vous ?
- Ouvrez un fichier HTML et analyser les résultats.

## MultiQC

L'outil `fastqc` produit une sortie HTML par fichier fastq. Analyser *un par un* chacun des fichiers peut être trèèèèès long, surtout quand on a beaucoup d'échantillons RNA-seq.
L'outil `multiqc` est un outil très utile qui permet d'agréger des sorties de beaucoup d'outils différents et générer un fichier HTML.

Commencez par jeter un coup d'œil aux différentes options du programme.

In [ ]:
multiqc -h

Parmi les options intéressantes, l'option `-i`permet de donner un nom au fichier de sortie. C'est très pratique pour garder une tracer de ce qu'on a fait (ça évite les noms de fichiers type `toto1.html`, `toto2.html`, etc...)

In [ ]:
multiqc -f -o ./Analysis_chr7/1-beforeTrimming/multiqc/ \
    -i "GSE76872 - FastQC before trimming" \
    --interactive \
    ./Analysis_chr7/1-beforeTrimming/fastqc/

# 3. Nettoyage des reads

## FastP

La deuxième étape d'un pipeline d'analyse RNA-seq est de nettoyer les reads de séquençage. Voici ce qu'on cherche à nettoyer : 
- Les reads / parties de reads de mauvaise qualité
- les partie de reads contenant des adapteurs (séquences *techniques* collées aux séquences *biologiques* qui permettent le séquençage)
- les reads trop courts

Le programme `fastp` permet de faire tout cela en une fois, de manière (quasi-)automatique.

Commencez par jeter un coup d'œil aux différentes options du programme.

In [ ]:
fastp --help

Comme vous pouvez le voir, il y a un certain nombre d'options (on ne va pas toutes les explorer).

Mais avant de se lancer, posons-nous quelques questions importantes :
- Combien d'échantillons différents avons-nous ?
- Combien de fichier(s) fastq par échantillon avons-nous ?
- En quoi cela va-t-il affecter nos analyses ?
- Comment faire tourner `fastp` (ou tout autre programme) en prenant cela en compte ?

Le fichier `samples.chr7.txt` contient la liste des différents échantillons. À votre avis, que permet de faire le code contenu dans la cellule ci-dessous ?

In [ ]:
while read line
do
    echo -e $line
done < /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/samples.chr7.txt

Chaque ligne du fichier d'entrée est donc enregistrée dans une variable (nommée `$line`) que l'on va pouvoir manipuler / utiliser.

Le premier réflexe serait de simplement compléter le nom de la variable pour former un nom de fichier, comme ci-dessous.

Faites-tourner la cellule pour voir si vous formez un nom de fichier fastq correct.

In [ ]:
while read line
do
    echo -e $line_chr7_R1.fastq.gz
done < /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/samples.chr7.txt

Normalement, la cellule ci-dessus ne devrait pas fonctionner (en tout cas pas donner des noms de fichiers corrects). `BASH` permet de manipuler les variables et le texte qu'elles contiennent en encadrant le nom de la variable avec des accolades (avec le `$` à l'extérieure des accolades) : 

`$line` --> `${line}`

Le code de la cellule ci-dessous devrait afficher des noms de fichiers corrects.

In [ ]:
while read line
do
    echo -e ${line}_chr7_R1.fastq.gz
done < /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/samples.chr7.txt

On peut donc afficher en mode détaillé chaque fichier de reads forward associé à chaque échantillon :

In [ ]:
while read line
do
    ls -alh /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/${line}_chr7_R1.fastq.gz
done < /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/samples.chr7.txt

Revenons à `fastp`. Comme vous l'avez vu, il y a un très grand nombre d'options disponibles pour cet outil. Dans le code ci-dessous, voici les options que l'on utilise :
- `--in1` & `--in2` : permet de spécifier les fichiers d'entrée *forward* et *reverse*
- `--out1` & `--out2` : permet de spécifier les fichiers de sortie *forward* et *reverse*
- `--json` : enregistre les informations de sortie au format *json* 
- `--json` : enregistre les informations de sortie au format *html*
- `--cut_right` : une des façons qu'à fastp de lire les reads (ici, par la partie 3')
- `--cut_window_size` : analyse les reads par fenêtre de *x* nucléotides
- `--cut_mean_quality` : spécifie la qualité moyenne *minimum* dans cette fenêtre
- `--length_required` : spécifie la longueur minimum d'un read
- `--detect_adapter_for_pe` : détection automatique d'adapteurs pour les reads *paired-end*
- `--dedup`: déduplication des reads (duplication PCR)
- `--dup_calc_accuracy` : précision de détection des reads dupliqués
- `-p` : analyse de sur-représentation
- `-P` : échantillonage pour l'analyse de sur-représentation
- `--thread` : spécifie le nombre de CPU avec lequel le programme va tourner
- `--verbose` : spécifie qu'on veut plus d'informations de sortie que par défaut. 

In [ ]:
while read line
do
    fastp --in1 /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/${line}_chr7_R1.fastq.gz \
    --in2 /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/${line}_chr7_R2.fastq.gz \
    --out1 ./Analysis_chr7/2-afterTrimming/fastp/${line}_chr7_R1.fastp.fastq.gz \
    --out2 ./Analysis_chr7/2-afterTrimming/fastp/${line}_chr7_R2.fastp.fastq.gz \
    --json ./Analysis_chr7/2-afterTrimming/fastp/${line}.json \
    --html ./Analysis_chr7/2-afterTrimming/fastp/${line}.html \
    --cut_right \
    --cut_window_size 5 \
    --cut_mean_quality 25 \
    --average_qual 25 \
    --length_required 20 \
    --detect_adapter_for_pe \
    --dedup \
    --dup_calc_accuracy 3 \
    -p -P 500 \
    --thread 5 \
    --verbose

done < /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/samples.chr7.txt

In [16]:
# multiqc -f -o ./Analysis_chr7/2-afterTrimming/multiqc/ \
#    -i "GSE76872 - trimming" \
#    --interactive \
#    ./Analysis_chr7/2-afterTrimming/fastp/*.json

## FastQC & MultiQC

Une fois que `fastp` a terminé de tourner, il faut aller voir les effets de ce programme sur nos données avec `fastqc` et `multiqc`.  

In [ ]:
fastqc -t 5 -o ./Analysis_chr7/2-afterTrimming/fastqc/ \
    ./Analysis_chr7/2-afterTrimming/fastp/*.fastq.gz

multiqc -f -o ./Analysis_chr7/2-afterTrimming/multiqc/ \
    -i "GSE76872 - FastQC after trimming" \
    --interactive \
    ./Analysis_chr7/2-afterTrimming/fastqc/

- Que pouvez-vous dire du résultat de `fastp` ? 
- Est-ce que beaucoup de séquences ont été enlevées ?

# 4. Alignement des reads

Une fois que les reads de séquençage ont été nettoyés, la troisième étape consiste à la cartographie (ou *mapping*) de ces reads *nettoyés* sur le génome d'intérêt.
Il existe un grand nombre d'outils faisant cela (*bowtie*, *bwa*, *HISAT2*, *STAR*), ici, c'est l'outil *STAR* que l'on va utiliser. 

Une de ses particularités est qu'il est capable de couper les reads en plusieurs morceaux et d'aligner chacuns des morceaux.
- En quoi cela est utile, voire nécessaire dans le cadre d'une analyse de RNA-seq ?
- Pourquoi est-ce que d'autres outils ne faisant pas cela ne seront pas adaptés

Un point important est que ces programmes ne peuvent pas travailler directement avec le génome de référence au format fasta (classique), mais on besoin d'*indéxer* le génome dans un format particulier. Cette étape a déjà été effectuée pour vous, mais gardez cette nécessité en tête.

Un autre point important est que ces programmes sont très gourmands en ressources informatique (CPU, RAM) et ne sont pas fait pour tourner sur des ordinateurs personnels, mais plutôt sur des clusters de calcul.

Commencez par regarder les options disponibles pour `STAR`

In [ ]:
STAR -h

Comme vous pouvez le voir, le nombre d'options disponibles est énorme. dans le code ci-dessous, voici les options que l'on utilise : 
- `--genomeDir` : le dossier contenant le génome indexé
- `--runThreadN` : le nombre de CPU
- `--readFilesIn` : les fichiers contenant les reads de séquençage
- `--outFileNamePrefix` : le préfixe (début du nom) pour les fichiers de sortie
- `--outSAMtype` : le format de sortie (ici au format BAM)

In [ ]:
while read line
do
    STAR --genomeDir /srv/data/Genomes/Mmu/GRCm39/indexes_chr7_upto49bases/ \
    --runThreadN 10 \
    --readFilesCommand zcat \
    --readFilesIn ./Analysis_chr7/2-afterTrimming/fastp/${line}_chr7_R1.fastp.fastq.gz ./Analysis_chr7/2-afterTrimming/fastp/${line}_chr7_R2.fastp.fastq.gz \
    --outFileNamePrefix ./Analysis_chr7/3-mapping/bam/${line}.chr7_ \
    --outSAMtype BAM SortedByCoordinate \

done < /srv/data/meg-m1-a2b/blumenthal-2014/chr7/1-fastq/samples.chr7.txt

**Questions** :
- Combien de fichiers de sortie par échantillons produit `STAR` ?
- Dans quel fichier peut-on trouver les informations de mapping (pourcentage de reads mappés) ?

## MultiQC

Une fois le mapping terminé pour tous les échantillons, on peut lancer l'outil `multiqc` pour agréger les informations de mapping.

In [ ]:
multiqc -f -o ./Analysis_chr7/3-mapping/multiqc/ \
    -i "GSE76872 - Mapping" \
    --interactive \
    ./Analysis_chr7/3-mapping/bam/

**Questions** : 
- Quelles informations pouvez-vous avoir sur la qualité du mapping ? Quelles informations vous semblent les plus importantes ?
- Que pouvez-vous dire sur la qualité globale du mapping ?

## Indexation des fichiers de mapping

Une fois le mapping lancé, on lance l'outil `samtools` pour indexer les fichiers de mapping. Cette étape est nécessaire pour des outils ultérieurs (*e.x.* IGV).

In [21]:
samtools index -M ./Analysis_chr7/3-mapping/bam/*.bam